# Exercises: DMD

Exercise on the implementation of Dynamic mode decomposition

Author: Stefano Pagani <stefano.pagani@polimi.it>.

Date: 2024

Course: Mathematical and numerical foundations of scientific machine learning.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import io
import os

import scipy
import scipy.integrate


In [ ]:
x1 = np.linspace(-3, 3, 80)
x2 = np.linspace(-3, 3, 80)
x1grid, x2grid = np.meshgrid(x1, x2)

time = np.linspace(0, 6, 16)

data = [1 / np.cosh(x1grid) / np.cosh(x2grid) * (1.2j**-t) + 1.0* np.cos(x1grid) * np.sin(x2grid) * (1.8j**-t) for t in time]
noise = [np.random.normal(0.0, 0.2, size=x1grid.shape) for t in time]

snapshots = [d + n for d, n in zip(data, noise)]

fig = plt.figure(figsize=(18, 18))
for id_subplot, snapshot in enumerate(snapshots, start=1):
    plt.subplot(4, 4, id_subplot)
    plt.pcolor(x1grid, x2grid, snapshot.real, vmin=-1, vmax=1)
    plt.axis('equal')

In [ ]:
# snapshot matrix
X = np.matrix( [ (snapshots[i]).flatten() for i in np.arange(0,16,1) ] ).T
np.shape(X)

#print(np.arange(0,16,1))

Task: implement the DMD passages

In [ ]:

def DMD(X,Xprime,r):

    # step 1: implement SVD of the snapshot matrix
    # U,Sigma,VT = 

    # truncate
    # Ur = 
    # Sigmar =
    # VTr = 

    # visualize singular values decay
    # plt.semilogy( ... )
    
    # step 2: compute Atilde

    # Atilde =  ...
    
    # step 3: perform spectral decomposition
    #Lambda, W = ...
    
    #step 4: compute the DMD eigen vector

    #Phi =  ...

    # matrix for prediction
    alpha1 = Sigmar @ VTr[:,0]
    b = np.linalg.solve(W @ Lambda,alpha1)
    return Phi, Lambda, b

Phi, Lambda, b = DMD(X[:,:-2],X[:,1:-1],1)



In [ ]:


prediction = np.zeros((6400,32),dtype = 'complex_')

A = (Phi @ Lambda @ np.linalg.pinv(Phi))

print('Guess')

for i in range(32):
    if i == 0:
        #prediction[:,i:i+1] = X[:,0]
        prediction[:,i:i+1] = Phi @  b
    else:
        #prediction[:,i:i+1] =  A @ prediction[:,i-1:i]
        prediction[:,i:i+1] = Phi @ (Lambda)**(i) @ b

fig = plt.figure(figsize=(18, 18))
for id_subplot, snapshot in enumerate(snapshots, start=1):
    #print(id_subplot)
    plt.subplot(4, 4, id_subplot)
    plt.pcolor(x1grid, x2grid, np.reshape(prediction[:,id_subplot-1].real,(80,80)) , vmin=-1, vmax=1)
    plt.axis('equal')



In [ ]:

compute_integral = scipy.integrate.trapz

original_int = [
    compute_integral(compute_integral(snapshot)).real for snapshot in snapshots
]
dmd_int = [
    compute_integral(compute_integral(np.reshape(prediction[:,i],(80,80)))).real for i in range(32)
]

figure = plt.figure(figsize=(18, 5))
plt.plot( original_int, "bo", label="original snapshots")
plt.plot( dmd_int, "r.", label="dmd states")
plt.ylabel("Integral")
plt.xlabel("Time")
plt.grid()
leg = plt.legend()
